In [ ]:
# install the package
%pip install --upgrade mani_skill
# install a version of torch that is compatible with your system
%pip install torch torchvision torchaudio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Command Line Options for panda:

- `-e, --env-id`, type=$str$, default="PickCube-v1", help=f"Environment to run motion planning solver on
- `-o, --obs-mode`, type=$str$, default="none", help="Observation mode to use. Usually this is kept as 'none' as observations are not necesary to be stored, they can be replayed later via the mani_skill.trajectory.replay_trajectory script.
- `-n, --num-traj`, type=$int$, default=10, help="Number of trajectories to generate."
- `--only-count-success`, action="store_true", help="If true, generates trajectories until num_traj of them are successful and only saves the successful trajectories/videos"
- `--reward-mode`, type=$str$
- `-b, --sim-backend`, type=$str$, default="auto", help="Which simulation backend to use. Can be 'auto', 'cpu', 'gpu'"
- `--render-mode`, type=$str$, default="rgb_array", help="can be 'sensors' or 'rgb_array' which only affect what is saved to videos"
- `--vis`, action="store_true", help="whether or not to open a GUI to visualize the solution live"
- `--save-video`, action="store_true", help="whether or not to save videos locally"
- `--traj-name`, type=$str$, help="The name of the trajectory .h5 file that will be created."
- `--shader`, default="default", type=$str$, help="Change shader used for rendering. Default is 'default' which is very fast. Can also be 'rt' for ray tracing and generating photo-realistic renders. Can also be 'rt-fast' for a faster but lower quality ray-traced renderer"
- `--record-dir`, type=$str$, default="demos", help="where to save the recorded trajectories"

# Environments

- PickCube-v1
- StackCube-v1
- PegInsertionSide-v1
- PlugCharger-v1
- PushCube-v1

In [ ]:

!python -m mani_skill.examples.motionplanning.panda.run -e "PickCube-v1"\
    --num-traj 100 \
    --only-count-success \ 
    --record-dir /content/drive/MyDrive/Data/Training/Generated \
    --traj-name data100 #File name is data100.h5
    
# The output will be saved in two files named data100.h5 and data100.json

# Command Line Options for replay:

- `--save-traj`: save the replayed trajectory to the same folder as the original trajectory file.
- `--target-control-mode`: The target control mode / action space to save into the trajectory file.
- `--save-video`: Whether to save a video of the replayed trajectories
- `--max-retry`: Max number of times to try and replay each trajectory
- `--discard-timeout`: Whether to discard trajectories that time out due to the default environment's max episode steps config
- `--allow-failure`: Whether to permit saving failed trajectories
- `--vis`: Whether to open the GUI and show the replayed trajectories on a display
- `--use-first-env-state`: Whether to use the first environment state of the given trajectory to initialize the environment
- `--num-procs=10`: split trajectories to multiple processes (e.g., 10 processes) for acceleration. Note this is done via CPU parallelization, not GPU. This argument is also currently incompatible with using the GPU simulation to replay trajectories.
- `--obs-mode=none`: specify the observation mode as `none`, i.e. not saving any observations.
- `--obs-mode=rgbd`: (not included in the script above) specify the observation mode as `rgbd` to replay the trajectory. If `--save-traj`, the saved trajectory will contain the RGBD observations.
- `--obs-mode=pointcloud`: (not included in the script above) specify the observation mode as `pointcloud`. We encourage you to further process the point cloud instead of using this point cloud directly (e.g. sub-sampling the pointcloud)
- `--obs-mode=state`: (not included in the script above) specify the observation mode as `state`
- `--use-env-states`: For each time step $t$, after replaying the action at this time step and obtaining a new observation at $t+1$, set the environment state at time $t+1$ as the recorded environment state at time $t+1$. This is necessary for successfully replaying trajectories for the tasks migrated from ManiSkill1.
- `--count`: Number of demonstrations to replay before exiting. By default all demonstrations are replayed
- `--shader`: "Change shader used for rendering. Default is 'default' which is very fast. Can also be 'rt' for ray tracing and generating photo-realistic renders. Can also be 'rt-fast' for a faster but lower quality ray-traced renderer"
- `--render-mode`: The render mode used in the video saving
- `-b, --sim-backend`: Which simulation backend to use. Can be 'auto', 'cpu', or 'gpu'


In [ ]:

!python -m mani_skill.trajectory.replay_trajectory \
  --traj-path /content/drive/MyDrive/Data/Training/Generated/PickCube-v1/data100.h5 \
  --save-traj \
  --obs-mode rgbd \
  --sim-backend gpu

# The output will be saved in two files named data100.pointcloud.h5 and data100.pointcloud.json